In [20]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

In [77]:
df = pd.read_csv('cleaned_data_0.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26664 entries, 0 to 26663
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   ID                          26664 non-null  object
 1   LIMIT_BAL                   26664 non-null  int64 
 2   SEX                         26664 non-null  int64 
 3   EDUCATION                   26664 non-null  int64 
 4   MARRIAGE                    26664 non-null  int64 
 5   AGE                         26664 non-null  int64 
 6   PAY_1                       26664 non-null  int64 
 7   PAY_2                       26664 non-null  int64 
 8   PAY_3                       26664 non-null  int64 
 9   PAY_4                       26664 non-null  int64 
 10  PAY_5                       26664 non-null  int64 
 11  PAY_6                       26664 non-null  int64 
 12  BILL_AMT1                   26664 non-null  int64 
 13  BILL_AMT2                   26664 non-null  in

In [82]:
Non_features = ['ID','default payment next month','PAY_5','PAY_6','BILL_AMT2','BILL_AMT3','BILL_AMT4','BILL_AMT5','BILL_AMT6']
X = df.drop(columns=Non_features).values
Y = df.iloc[:,24].values

In [28]:
scale = StandardScaler()

X_scale = scale.fit_transform(X)
Y_scale = Y.ravel()

print(X_scale.shape,Y_scale.shape)

(26664, 29) (26664,)


In [83]:
from sklearn.preprocessing import RobustScaler

scale = RobustScaler()
X_scale = scale.fit_transform(X)
Y_scale = Y.ravel()

print(X_scale.shape,Y_scale.shape)

(26664, 16) (26664,)


In [91]:
X_train, X_test, Y_train, Y_test = train_test_split(X_scale, Y_scale, test_size=0.2, random_state=42)

In [118]:
clf = RandomForestClassifier(n_estimators=100)

# use a full grid over all parameters
param_grid = {"max_depth": [3, None],
              "max_features": [1, 3, 10],
              "min_samples_split": [1, 3, 10],
              "min_samples_leaf": [1, 3, 10],
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]}

In [125]:
RF_clf = GridSearchCV(clf, param_grid=param_grid, n_jobs=-1, cv=5)
RF_clf = RF_clf.fit(X_train, Y_train)
print('The best model:\n', grid_search.best_params_)

/Users/haoli/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
360 fits failed out of a total of 1080.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
180 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/haoli/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/haoli/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_forest.py", line 450, in fit
    trees = Parallel(
  File "/Users/haoli/opt/anaconda3/lib/python3.8/site-packages/joblib/parallel.py", line 1043, in __call__
    if self.dispatch_one_batch(iterator):
 

The best model:
 {'C': 0.1, 'fit_intercept': True, 'penalty': 'l2'}


In [126]:
RF_clf = RF_clf.best_estimator_
print(RF_clf.score(X_test,Y_test))

0.8261766360397524


In [85]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from keras.layers import Dropout


model = Sequential()
model.add(Dense(5, input_dim = len(X_train[0, :]), activation = 'relu'))

model.add(Dense(3, activation = 'relu'))


model.add(Dense(1, activation = 'sigmoid'))
model.summary()

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_44 (Dense)             (None, 5)                 85        
_________________________________________________________________
dense_45 (Dense)             (None, 3)                 18        
_________________________________________________________________
dense_46 (Dense)             (None, 1)                 4         
Total params: 107
Trainable params: 107
Non-trainable params: 0
_________________________________________________________________


In [86]:
from tensorflow import keras
opt = keras.optimizers.Adam(learning_rate=0.0005)
model.compile(loss = 'mean_absolute_error',optimizer = opt, metrics=['accuracy'])
model.fit(X_train, Y_train, validation_data =(X_test, Y_test), epochs = 20, batch_size = 3 )

Epoch 1/20
7111/7111 [==============================] - 7s 911us/step - loss: 0.2360 - accuracy: 0.7757 - val_loss: 0.2114 - val_accuracy: 0.7844
Epoch 2/20
7111/7111 [==============================] - 6s 828us/step - loss: 0.2119 - accuracy: 0.7875 - val_loss: 0.2010 - val_accuracy: 0.8071
Epoch 3/20
7111/7111 [==============================] - 6s 847us/step - loss: 0.1997 - accuracy: 0.8067 - val_loss: 0.1925 - val_accuracy: 0.8106
Epoch 4/20
7111/7111 [==============================] - 6s 845us/step - loss: 0.1934 - accuracy: 0.8092 - val_loss: 0.1893 - val_accuracy: 0.8134
Epoch 5/20
7111/7111 [==============================] - 6s 847us/step - loss: 0.1906 - accuracy: 0.8108 - val_loss: 0.1870 - val_accuracy: 0.8136
Epoch 6/20
7111/7111 [==============================] - 6s 832us/step - loss: 0.1874 - accuracy: 0.8136 - val_loss: 0.1846 - val_accuracy: 0.8172
Epoch 7/20
7111/7111 [==============================] - 6s 807us/step - loss: 0.1855 - accuracy: 0.8156 - val_loss: 0.1825 -

KeyboardInterrupt: 

In [48]:
prediction = model.predict(X_test)

In [51]:
print(prediction)

[[0.17946249]
 [0.1403653 ]
 [0.09882686]
 ...
 [0.08635059]
 [0.09199658]
 [0.5151571 ]]


In [87]:
def update_weights_sgd(X_train, y_train, weights, learning_rate):
    for X_each, y_each in zip(X_train, y_train):
        prediction = compute_prediction(X_each, weights)
        weights_delta = X_each.T * (y_each - prediction)
        weights += learning_rate * weights_delta
    return weights

def train_logistic_regression_sgd(X_train, y_train, max_iter, learning_rate, fit_intercept=False):
    if fit_intercept:
        intercept = np.ones((X_train.shape[0], 1))
        X_train = np.hstack((intercept, X_train))
    weights = np.zeros(X_train.shape[1])+0.5
    for iteration in range(max_iter):
        weights = update_weights_sgd(X_train, y_train, weights, learning_rate)
        # Check the cost for every 2 (for example) iterations
        if iteration % 2 == 0:
            print(compute_cost(X_train, y_train, weights))
    return weights

def sigmoid(input):
    return 1.0 / (1 + np.exp(-input))

def compute_prediction(X, weights):
    z = np.dot(X, weights)
    predictions = sigmoid(z)
    return predictions

def compute_cost(X, y, weights):
    predictions = compute_prediction(X, weights)
    cost = np.mean(-y * np.log(predictions) - (1 - y) * np.log(1 - predictions))
    return cost

def predict(X, weights):
    if X.shape[1] == weights.shape[0] - 1:
        intercept = np.ones((X.shape[0], 1))
        X = np.hstack((intercept, X))
    return compute_prediction(X, weights)

def classification(threshold_P, X, weights):
    vec = predict(X, weights)
    vec = np.where(vec > threshold_P, 1, 0)
    return vec

In [96]:
weights = train_logistic_regression_sgd(X_train, Y_train, 200, 0.0001, fit_intercept=True)

0.5691444593641837
0.5098160404281771
0.4974329790203034
0.4905266530686821
0.4857806973998171
0.48222626783107414
0.4794546914108589
0.4772482367192475
0.47547116810503115
0.4740298215156429
0.4728553323250336
0.4718950922358343
0.4711079812587019
0.47046140700723865
0.469929304128116
0.4694906990905917
0.4691286392026955
0.4688293730589745
0.46858171313635105
0.46837653464283896
0.46820637845707314
0.46806513471988004
0.46794778953736976
0.46785022143044264
0.4677690372183409
0.46770143930638985
0.4676451180817761
0.46759816445310864
0.4675589986014872
0.4675263118157347
0.4674990189157099
0.4674762192649602
0.4674571647673962
0.467441233555022
0.4674279083225677
0.467416758463701
0.4674074253228417
0.4673996100046665
0.46739306328659597
0.46738757726292346
0.467382978416764
0.4673791218708041
0.46737588661241114
0.4673731715250028
0.4673708920872518
0.46736897762599255
0.46736736902859594
0.4673660168369266
0.4673648796584334
0.46736392284099015
0.46736311736723
0.46736243893164675


In [99]:
predictions = classification(0.3,X_test,weights)
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
print(confusion_matrix(Y_test, predictions))
print(accuracy_score(Y_test, predictions))

[[3782  401]
 [ 622  528]]
0.8081755109694356
